In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
import math
import pandas as pd
import os
import re
import random

print("=" * 80)
print("폴란드 아마존 스킨케어 검색결과 1차 크롤링 (Selenium 버전)")
print("=" * 80)

# 👉 1. 설정
query_txt = '폴란드_스킨케어'
target_category = '스킨케어'

# 👉 폴란드 아마존 검색 URL
query_url = "https://www.amazon.pl/s?i=beauty&rh=n%3A20805443031&s=popularity-rank&fs=true"
base_url = "https://www.amazon.pl"

# 수집 건수
cnt = int(input('크롤링 할 건수는 몇건입니까? (예: 50): '))
page_cnt = math.ceil(cnt / 48)

# 👉 저장 경로
f_dir = input('파일을 저장할 폴더명(기본값: c:\\py_temp\\): ')
if f_dir == '':
    f_dir = "c:\\py_temp\\"

# 타임스탬프
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (
    now.tm_year, now.tm_mon, now.tm_mday,
    now.tm_hour, now.tm_min, now.tm_sec
)

# 작업 폴더
work_dir = os.path.join(f_dir, s + '-' + query_txt)
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

fx_name = os.path.join(work_dir, f'{s}-{query_txt}.xlsx')

# 크롬 옵션
options = Options()
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
)
options.add_argument("--lang=ko-KR")
# options.add_argument("--headless")  # 백그라운드 실행 (필요시)

# 크롬 드라이버 실행
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)
driver.get(query_url)
time.sleep(random.uniform(3, 5))
driver.maximize_window()
time.sleep(random.uniform(2, 4))

# 스크롤 함수
def scroll_down():
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(random.uniform(1, 2))

# ASIN 추출 함수
def extract_asin(url):
    try:
        match = re.search(r'/dp/([A-Z0-9]{10})', url)
        if match:
            return match.group(1)
        return ""
    except:
        return ""

# 👉 결과 저장 리스트
result_data = {
    'country': [],
    'platform': [],
    'category_mid': [],
    'product_id': [],
    'rank': [],
    'product_name': [],
    'product_detail': [],
    'price': [],
    'review_count': [],
    'rating': [],
    'product_url': []
}

total_cnt = 0

for page_num in range(1, page_cnt + 1):
    
    if total_cnt >= cnt:
        break
    
    print(f"\n{'='*60}")
    print(f"페이지 {page_num} 크롤링 중...")
    print(f"{'='*60}\n")
    
    # 스크롤
    for _ in range(5):
        scroll_down()
    
    time.sleep(2)
    
    # 👉 Selenium으로 직접 상품 카드 찾기
    try:
        product_cards = driver.find_elements(By.CSS_SELECTOR, 
            "div[data-component-type='s-search-result']"
        )
        
        if not product_cards:
            print("⚠️ 상품 리스트를 찾지 못했습니다.")
            break
        
        print(f"  → {len(product_cards)}개 상품 발견")
        
    except Exception as e:
        print(f"⚠️ 상품 카드 로딩 실패: {e}")
        break
    
    for card in product_cards:
        
        if total_cnt >= cnt:
            break
        
        try:
            print(f"\n[{total_cnt + 1}/{cnt}] 수집 중...")
            
            # ASIN
            asin = card.get_attribute('data-asin')
            if not asin:
                continue
            
            print(f"  → ASIN: {asin}")
            
            # URL
            url = f"{base_url}/dp/{asin}"
            
            # 👉 제품명 (Selenium으로 직접 찾기)
            title = ""
            try:
                # 방법1: h2 안의 텍스트
                h2_elem = card.find_element(By.CSS_SELECTOR, "h2")
                title = h2_elem.text.strip()
            except:
                pass
            
            if not title:
                try:
                    # 방법2: a 태그
                    link_elem = card.find_element(By.CSS_SELECTOR, "h2 a")
                    title = link_elem.text.strip()
                except:
                    pass
            
            if not title:
                try:
                    # 방법3: span 태그들 중에서
                    spans = card.find_elements(By.CSS_SELECTOR, "span")
                    for span in spans:
                        text = span.text.strip()
                        if len(text) > 15 and 'zł' not in text and ',' not in text[:5]:
                            title = text
                            break
                except:
                    pass
            
            if not title or len(title) < 5:
                print("  ⚠️ 제품명 없음 → 건너뜀")
                continue
            
            print(f"  → 제품명: {title[:50]}...")
            
            # 👉 가격
            price = 0
            try:
                price_elem = card.find_element(By.CSS_SELECTOR, "span.a-price-whole")
                price_text = price_elem.text.strip().replace(',', '.')
                # decimal 부분 제거
                price_text = re.sub(r'[^\d.]', '', price_text)
                price = float(price_text)
            except:
                pass
            
            # 👉 별점
            rating = 0
            try:
                # 숫자로 표시된 별점
                rating_elem = card.find_element(By.CSS_SELECTOR, "span.a-icon-alt")
                rating_text = rating_elem.get_attribute('innerHTML')
                m = re.search(r"(\d+[.,]\d+)", rating_text)
                if m:
                    rating = float(m.group(1).replace(",", "."))
            except:
                pass
            
            # 👉 리뷰 개수
            review_count = 0
            try:
                review_elem = card.find_element(By.CSS_SELECTOR, "span[aria-label*='ocen']")
                review_text = review_elem.get_attribute('aria-label')
                review_count = int(re.sub(r'[^\d]', '', review_text))
            except:
                try:
                    # 대체 방법
                    review_elem = card.find_element(By.XPATH, ".//span[contains(text(), '(')]")
                    review_text = review_elem.text
                    review_count = int(re.sub(r'[^\d]', '', review_text))
                except:
                    pass
            
            print(f"  → 가격: {price} PLN | 별점: {rating} | 리뷰: {review_count}개")
            
            # 👉 상세내용 (상세 페이지 접근)
            product_detail = ""
            try:
                # 새 탭으로 열기
                driver.execute_script("window.open(arguments[0]);", url)
                time.sleep(random.uniform(2, 3))
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(random.uniform(2, 3))
                
                current_url = driver.current_url
                
                if ("signin" not in current_url) and ("ap/cvf" not in current_url):
                    # 👉 상세페이지에서 제목으로 상세내용 찾기
                    try:
                        detail_elem = driver.find_element(By.ID, "feature-bullets")
                        product_detail = detail_elem.text.strip()
                    except:
                        try:
                            detail_elem = driver.find_element(By.ID, "productDescription")
                            product_detail = detail_elem.text.strip()
                        except:
                            pass
                    
                    if product_detail:
                        print(f"  → 상세내용: {product_detail[:60]}...")
            except Exception as e:
                print(f"  ⚠️ 상세 페이지 에러: {str(e)[:50]}")
            
            # 탭 닫기
            try:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(random.uniform(1, 2))
            except:
                pass
            
            # 👉 데이터 저장
            result_data['country'].append('PL')
            result_data['platform'].append('Amazon')
            result_data['category_mid'].append(target_category)
            result_data['product_id'].append(asin)
            result_data['rank'].append(total_cnt + 1)
            result_data['product_name'].append(title)
            result_data['product_detail'].append(product_detail[:500])
            result_data['price'].append(price)
            result_data['review_count'].append(review_count)
            result_data['rating'].append(rating)
            result_data['product_url'].append(url)
            
            total_cnt += 1
            
            # 50개마다 휴식
            if total_cnt % 50 == 0:
                print(f"\n{'='*60}")
                print(f"✅ {total_cnt}개 수집 완료! 30초 휴식...")
                print(f"{'='*60}\n")
                time.sleep(30)
            
            time.sleep(random.uniform(1, 2))
            
        except Exception as e:
            print(f"  ⚠️ 상품 처리 중 에러: {str(e)[:100]}")
            continue
    
    if total_cnt >= cnt:
        break
    
    # 다음 페이지
    try:
        time.sleep(random.uniform(3, 4))
        next_btn = driver.find_element(By.CSS_SELECTOR, "a.s-pagination-next")
        driver.execute_script("arguments[0].click();", next_btn)
        print(f"\n→ 다음 페이지로 이동\n")
        time.sleep(random.uniform(4, 6))
    except:
        print("⚠️ 다음 페이지 없음")
        break

# DataFrame 생성
df = pd.DataFrame(result_data)

# 중복 제거
df = df.drop_duplicates(subset=['product_id'])

# 엑셀 저장
df.to_excel(fx_name, index=False, engine='openpyxl')

print("\n" + "="*80)
print("✅ 크롤링 완료!")
print(f"💾 저장 파일: {fx_name}")
print(f"📊 총 수집: {len(df)}개 제품")

if len(df) > 0:
    print(f"   - 평균 가격: {df['price'].mean():.2f} PLN")
    print(f"   - 평균 평점: {df['rating'].mean():.2f}")
    print(f"   - 평균 리뷰: {df['review_count'].mean():.0f}개")
    print(f"   - 리뷰 0개: {len(df[df['review_count']==0])}개")
    
    print("\n🏆 Top 5 제품:")
    for i, row in df.head(5).iterrows():
        print(f"  {row['rank']}. {row['product_name'][:50]}")
        print(f"     ASIN: {row['product_id']} | 평점: {row['rating']} | 리뷰: {row['review_count']}개\n")
else:
    print("\n⚠️ 수집된 데이터가 없습니다.")

print("="*80)

driver.quit()

폴란드 아마존 스킨케어 검색결과 1차 크롤링 (Selenium 버전)


크롤링 할 건수는 몇건입니까? (예: 50):  50
파일을 저장할 폴더명(기본값: c:\py_temp\):  



페이지 1 크롤링 중...

  → 51개 상품 발견

[1/50] 수집 중...
  → ASIN: B086W1ST56
  → 제품명: O'Keeffe's Working Hands, 96 g tygiel 2 sztuki – k...
  → 가격: 43.0 PLN | 별점: 4.7 | 리뷰: 15개
  → 상세내용: Informacje o tym produkcie
Pielęgnacja skóry: O'Keeffe's Wor...

[2/50] 수집 중...
  → ASIN: B09XN22JBT
  → 제품명: Cerave Am Spf 50 Krem do Twarzy, 52 ml...
  → 가격: 41.0 PLN | 별점: 4.6 | 리뷰: 143개
  → 상세내용: Informacje o tym produkcie
Proste i skuteczne działanie
Nieu...

[3/50] 수집 중...
  → ASIN: B07VPWLFYM
  → 제품명: L'Oréal Paris Men Expert Barber Club Żel myjący Br...
  → 가격: 19.0 PLN | 별점: 4.8 | 리뷰: 49개
  → 상세내용: Informacje o tym produkcie
Skóra twarzy jest oczyszczona, a ...

[4/50] 수집 중...
  → ASIN: B000VJXIVI
  → 제품명: Neutrogena Pielęgnacja ust, biały...
  → 가격: 9.0 PLN | 별점: 4.6 | 리뷰: 72개
  → 상세내용: Informacje o tym produkcie
Gładkie usta — The Neutrogena Bal...

[5/50] 수집 중...
  → ASIN: B0D2XYC4G2
  → 제품명: O'Keeffe's Working Hands, 96 g tygiel – krem do rą...
  → 가격: 25.0 PLN | 별점: 4.4 | 리뷰: 226개
  → 상세내용: Inform